In [ ]:
# xgboost
#https://xgboost.readthedocs.io/en/latest/python/index.html
# https://github.com/dmlc/xgboost/blob/master/demo/multiclass_classification/train.py#L35

In [1]:
!ls 

Dtree.ipynb                t10k-labels-idx1-ubyte.gz
RF.ipynb                   train-images-idx3-ubyte.gz
__pycache__                train-labels-idx1-ubyte.gz
load_mnist.py              xgboost.ipynb
t10k-images-idx3-ubyte.gz


In [15]:
from load_mnist import load_mnist
import xgboost as xgb
import numpy as np
from sklearn.preprocessing import StandardScaler
train_data, train_label = load_mnist("","train")
test_data,test_label = load_mnist("","t10k")

In [16]:

sc = StandardScaler()
X_main_std = sc.fit(train_data)

train_data = X_main_std.transform(train_data)
test_data = X_main_std.transform(test_data)


# min_max_scaler=preprocessing.MinMaxScaler()  
# min_max=min_max_scaler.fit(train_data)

# train_data = min_max.transform(train_data)
# test_data = min_max.transform(test_data)


/Users/geyang/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [17]:

# num_round = [2,10,50,150]
# param['max_depth'] = [2, 4, 6, 8]
# param['max_depth'] = [5, 20, 50, 100]
# param['max_depth'] = [2,...,15]

# param_list = [("eta", 0.08), ("max_depth", 6), ("subsample", 0.8), 
#               ("colsample_bytree", 0.8), ("objective", "multi:softmax"), 
#               ("eval_metric", "merror"), ("alpha", 8), ("lambda", 2), ("num_class", 10)]
param = {}
param['objective'] = 'multi:softmax'
param['eta'] = 0.3
param['num_class'] = 10
param["subsample"] = 0.8
param["colsample_bytree"] = 0.8
param["eval_metric"] = 'merror'
param['silent'] = 1

num_round = 100
param['max_depth'] = 5
print(param)
print(num_round)

{'objective': 'multi:softmax', 'eta': 0.3, 'num_class': 10, 'subsample': 0.8, 'colsample_bytree': 0.8, 'eval_metric': 'merror', 'silent': 1, 'max_depth': 5}
100


In [21]:
# bst = xgb.train(param, dtrain, num_round, evallist)

# bst = xgb.train(param, dtrain, num_round)
# for i in range(1):

import time
from sklearn.model_selection import train_test_split

# arr = np.arange(train_data.shape[0])
# np.random.shuffle(arr)
# num_of_train = int(arr.shape[0]*0.8)
# dtrain = xgb.DMatrix(train_data[arr[:num_of_train]], label=train_label[arr[:num_of_train]])
# dval = xgb.DMatrix(train_data[arr[num_of_train:]], label=train_label[arr[num_of_train:]])

x_train, x_valid, y_train, y_valid = train_test_split(train_data, train_label, test_size=0.2)

dtrain = xgb.DMatrix(x_train, label=y_train)
dval = xgb.DMatrix(x_valid, label=y_valid)

dtest = xgb.DMatrix(test_data, label=test_label)
evallist = [(dtrain, 'train'),(dval, 'validation')]



In [22]:
time1 = time.time()
early_stopping = 5
bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=early_stopping)
pred = bst.predict(dtest)
acc_rate = np.sum(pred == test_label) / test_label.shape[0]
print('Test accuracy using softmax = {}'.format(acc_rate))
time2 = time.time()
print((time2-time1)," seconds")

[0]	train-merror:0.189688	validation-merror:0.202417
Multiple eval metrics have been passed: 'validation-merror' will be used for early stopping.

Will train until validation-merror hasn't improved in 5 rounds.
[1]	train-merror:0.168583	validation-merror:0.182917
[2]	train-merror:0.16025	validation-merror:0.172833
[3]	train-merror:0.152208	validation-merror:0.168
[4]	train-merror:0.144917	validation-merror:0.16125
[5]	train-merror:0.140604	validation-merror:0.1565
[6]	train-merror:0.135417	validation-merror:0.15475
[7]	train-merror:0.131396	validation-merror:0.150917
[8]	train-merror:0.127813	validation-merror:0.147917
[9]	train-merror:0.122958	validation-merror:0.145417
[10]	train-merror:0.11925	validation-merror:0.141167
[11]	train-merror:0.115854	validation-merror:0.139167
[12]	train-merror:0.112042	validation-merror:0.137917
[13]	train-merror:0.108792	validation-merror:0.134917
[14]	train-merror:0.105979	validation-merror:0.13275
[15]	train-merror:0.103771	validation-merror:0.13183

In [ ]:
# #!/usr/bin/python
# import numpy as np
# import xgboost as xgb

# ### load data in do training
# param = {}
# # use softmax multi-class classification
# param['objective'] = 'multi:softmax'
# # scale weight of positive examples
# param['eta'] = 0.1
# param['max_depth'] = 10
# param['silent'] = 1
# param['nthread'] = 6
# param['num_class'] = 10
# num_round = 2

# xgb.cv(param, dtrain, num_round, nfold=5, metrics={'auc'}, seed=0)
